# Задание

1. Написать приложение, которое собирает основные новости с сайтов `mail.ru, lenta.ru, yandex.news`
2. Сложить все новости в БД

Для парсинга использовать xpath. Структура данных должна содержать:
- название источника,
- наименование новости,
- ссылку на новость,
- дата публикации

### Подключение библиотек

In [3]:
import requests
from pprint import pprint
from lxml import html
from pymongo import MongoClient
from datetime import datetime
import json

---

### Запуск сервера

In [2]:
try:
    client = MongoClient('localhost',27017)
    print('connected successfully')
except:
    print('bad connection')

db = client['database_news']  # создание БД
news_from_3_sites = db.news_from_3_sites  # содание коллекции

connected successfully!


---

In [4]:
main_link_mail = 'https://https://news.mail.ru'
main_link_ya = 'https://yandex.ru/news/'
main_link_lenta = 'https://lenta.ru'

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}

def tree(main_link):
    response = requests.get(main_link, headers=header).text
    tree = html.fromstring(response)
    return tree

In [ ]:
# - название источника,
# - наименование новости,
# - ссылку на новость,
# - дата публикации

In [ ]:
# news_all = []

### Сбор новостей

#### https://yandex.ru/news/

In [19]:
news_list_ya = tree(main_link_ya).xpath("//div[@class='story__topic']")

In [20]:
for news in news_list_ya:
    news_data={}
    
    name = str(news.xpath(".//h2[@class='story__title']/a/text()")[0])
    origin = str(news.xpath("..//div[@class='story__info']/div[1]/text()")[0])[:-5]
    href = 'https://yandex.ru' + str(news.xpath(".//h2[@class='story__title']/a/@href")[0])
    date_time = str(datetime.now())[:10] + str(news.xpath("..//div[@class='story__info']/div[1]/text()")[0])[-5:]    
    
    news_data['1 название источника'] = origin
    news_data['2 наименование новости'] = name
    news_data['3 ссылка на новость'] = href
    news_data['4 дата публикации'] = date_time
    
    news_from_3_sites.insert_one(news_data)

#### https://lenta.ru/

In [21]:
news_list_lenta = tree(main_link_lenta).xpath("//section[@class='row b-top7-for-main js-top-seven']/div[@class='span4']/div")

In [22]:
for news in news_list_lenta[:-1]:
    news_data={}
    
    name = str(news.xpath(".//a[1]/text()")[0])
    origin = main_link_lenta
    href = main_link_lenta + str(news.xpath("./a/@href")[0])
    date_time = str(news.xpath(".//a[1]/time[1]/@title")[0])
      
    news_data['1 название источника'] = origin
    news_data['2 наименование новости'] = name
    news_data['3 ссылка на новость'] = href
    news_data['4 дата публикации'] = date_time
    
    news_from_3_sites.insert_one(news_data)

#### https://news.mail.ru/

In [5]:
tree_mail = tree(main_link_mail)

ConnectionError: HTTPSConnectionPool(host='https', port=443): Max retries exceeded with url: //news.mail.ru (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000010D97DDA648>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
# news_list_mail_1 = tree_mail.xpath("//td[@class='daynews__main']//span[@class='photo__title photo__title_new photo__title_new_hidden js-topnews__notification']")
# # news_list_mail_3 = tree_mail.xpath("//a[@class='newsitem__title link-holder']")

In [9]:
# for news in news_list_mail:
#     news_data={}
    
#     name = ''
#     origin = ''
#     href = ''
#     date_time = ''
    
#     print(name)
#     break
# #     name = str(news.xpath(".//a[1]/text()")[0])
# #     origin = main_link_lenta
# #     href = main_link_lenta + str(news.xpath("./a/@href")[0])
# #     date_time = str(news.xpath(".//a[1]/time[1]/@title")[0])
      
# #     news_data['1 название источника'] = origin
# #     news_data['2 наименование новости'] = name
# #     news_data['3 ссылка на новость'] = href
# #     news_data['4 дата публикации'] = date_time
    
# #     news_from_3_sites.insert_one(news_data)

### Проверка

In [64]:
from pandas import DataFrame

cursor = news_from_3_sites.find({})
DataFrame(list(cursor)).drop(['_id'], axis=1)

,1 название источника,2 наименование новости,3 ссылка на новость,4 дата публикации
0,Коммерсантъ,В России выявили 33 новых случая заражения кор...,https://yandex.ru/news/story/V_Rossii_vyyavili...,2020-03-1818:51
1,Российская газета,«Евровидение-2020» официально отменили,https://yandex.ru/news/story/Evrovidenie-2020_...,2020-03-1819:00
2,Газета.Ru,Курс евро превысил 86 рублей впервые с 2016 года,https://yandex.ru/news/story/Kurs_evro_prevysi...,2020-03-1818:50
3,RT на русском,В Кремле прокомментировали слова Путина о сред...,https://yandex.ru/news/story/V_Kremle_prokomme...,2020-03-1818:53
4,РБК,Собянин попросил компании Москвы перевести сот...,https://yandex.ru/news/story/Sobyanin_poprosil...,2020-03-1818:51
...,...,...,...,...
70,https://lenta.ru,Вода в Венеции очистилась из-за коронавируса,https://lenta.ru/news/2020/03/18/venice/,18 марта 2020
71,https://lenta.ru,Невеста похвасталась обручальным кольцом и был...,https://lenta.ru/news/2020/03/18/badnails/,18 марта 2020
72,https://lenta.ru,Найдено место последнего боя летчика Филипова,https://lenta.ru/news/2020/03/18/place/,18 марта 2020
73,https://lenta.ru,В России появилась гречка за миллион рублей,https://lenta.ru/news/2020/03/18/million_rub_g...,18 марта 2020


---

При необходимости - очищение коллекции

In [ ]:
# news_from_3_sites.delete_many({})